In [2]:
import os
from time import time
from feature_extraction import __get

path = r'/Users/loewi/Documents/Pre_Learn/classification/20news-bydate/'
os.chdir(path)
#print(os.getcwd())

t0 = time() 

newsgroups_train = __get('20news-bydate-train')
newsgroups_test = __get('20news-bydate-test')

duration = time() - t0
print('%0.2fs get data package ：）'%duration)

X_train, X_test = newsgroups_train['data'], newsgroups_test['data']
y_train, y_test = newsgroups_train['docs'], newsgroups_test['docs']
print('X & y prepared')

2.95s get data package ：）
X & y prepared


In [5]:
from sklearn.feature_extraction.text import HashingVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from time import time


In [6]:
#调参
pipeline = Pipeline([
                ('vect', HashingVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b',
                        )),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(
                        
                        )),
                ])        
parameters = {
    'clf__solver': ('newton-cg', 'lbfgs','sag'),  
    'clf__multi_class':('ovr','multinomial'),  
    }   

cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = GridSearchCV(pipeline,parameters,n_jobs=-1, verbose=1,cv = cv)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)

t0 = time()
clf.fit(X_train, y_train)
m, s = divmod(time() - t0, 60)
h, m = divmod(m, 60)
print ("done in %02d:%02d:%02d" % (h, m, s))
print()

print("Best score: %0.3f" % clf.best_score_)
print("Best parameters set:")
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name])) 
       

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__multi_class': ('ovr', 'multinomial'),
 'clf__solver': ('newton-cg', 'lbfgs', 'sag')}
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 65.2min finished


done in 01:07:23

Best score: 0.799
Best parameters set:
	clf__multi_class: 'multinomial'
	clf__solver: 'newton-cg'


In [7]:
from sklearn import metrics  
pipeline_validated = Pipeline([
                ('vect', HashingVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b',
                        )),

                ('clf', LogisticRegression(
                        solver='newton-cg',  
                        multi_class='multinomial', 
                        )),
                ])    
print('Training...') 
pipeline_validated.fit(X_train, y_train) 

pred = pipeline_validated.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("testing accuracy:   %0.3f" % score)

Training...
testing accuracy:   0.689
